<center><h1>Using Grover's Algorithm in Battleship</h1></center>

Battleship is a beloved game enjoyed all over the world. Here we use Grover's Algorithm to solve for the possible configurations of a simplified version of Battleship on a 2 by 2 board.

<img src="https://user-images.githubusercontent.com/42923017/114302152-ce7c7680-9a95-11eb-97f8-e6a17c7144ea.png" alt="Richard's Battleship" width="300"/>

You are starting a round of Battleship against a friend, and you'd like to find all the different configurations you could place your ships. However, you must abide by the following rules:

- You have three ships with the following sizes: $1 \times 1$, $2 \times 1$ (call these ships "small" and "large", respectively.)

- The ships cannot overlap.

- The ship cannot be placed over the edge of a board.


<table style="width:100%">
    <tr>
        <td>
            <img src="https://user-images.githubusercontent.com/42923017/114302345-af321900-9a96-11eb-905b-7d0d029f22c8.png" alt="Richard's Invalid Battleship" width="300" caption="Invalid Battleship"/>
        </td>
        <td>
            <img src="https://user-images.githubusercontent.com/42923017/114302423-f3bdb480-9a96-11eb-8dab-07be87d86a7f.png" alt="Richard's Battleship" width="300" caption="Valid Battleship"/>
        </td>
    </tr>
</table>

On the left, we have an *invalid* battleship configuration because the $1\times 1$ ship in the lower left corner intersects with the $2\times 2$ ship on the bottom.

On the right, we have a *valid* battleship configuration because the $1\times 1$ ship in the upper left corner does not intersect with the $2\times 2$ ship on the right.

### Part 1. Setup

To help formalize the problem, we represent the board using a $2 \times 2$ graph on the $x$-$y$ plane, with each ship's coordinates represented by a length $2$ tuple of binary values $(x, y)$.

A configuration is uniquely specified by the positions of each of the two ships. Therefore, a configuration is characterized by the form

$$
(\text{small}_x, \text{small}_y), ([\text{large_start}_x, \text{large_start}_y], [\text{large_end}_x, \text{large_end}_y]),
$$

as demonstrated below:

<img src="https://user-images.githubusercontent.com/42923017/114303269-4305e400-9a9b-11eb-98ae-4943b1f27689.png" alt="Richard's Invalid Battleship" width="450" caption="Annotated Battleship"/>

This can be written as a qubit array of the form

$$
[\text{small}_x, \text{small}_y, \text{large_start}_x, \text{large_start}_y, \text{large_end}_x, \text{large_end}_y],
$$

which we use as our register for our marking oracle.

### Part 2. Implementation of Oracle

Given that the $1\times 1$ ship can start in any of the positions, our marking oracle only needs two ancillary qubits: one for checking the $2\times 1$ ship's validity (`validRectangle`), and one for checking the $2\times 1$ ship's overlap with the $1\times 1$ ship (`hasOverlap`).

We note that exactly one coordinate of the $2\times 1$ ship must be $[0, 0]$ or $[1, 1]$, so we use Controlled X to check for an instance of $[0, 0]$ or $[1, 1]$ in each coordinate and update the `validRectangle` target accordingly. The special case to consider is when both coordinates are $[0, 0]$ or $[1, 1]$, in which case `validRectangle` is targeted twice and thereby remains false, as desired.

Next, we check if the $1\times 1$ ship's coordinates overlap with either of the $2\times 2$ ships' coordinates by borrowing the 2 bit equality oracle from the Graph Coloring Katas. Since `hasOverlap` is initialized as `false`, it remains false when no overlap is detected. Therefore, our final `ControlledOnBitString` uses `[true, false]` as the control for `target`. 

In [79]:
// Import necessary packages

open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;


In [23]:
operation ValidBattleshipMarkingOracle (
    coordsRegister : Qubit[],
    target : Qubit
) : Unit is Adj+Ctl {
    // first ancilla qubit denotes if large is valid rectangle,
    // second qubit denotes if large overlaps with small
    use validRectangle = Qubit();
    use hasOverlap = Qubit();
    let (small_x, small_y, large_start_x, large_start_y, large_end_x, large_end_y) = (coordsRegister[0], coordsRegister[1], coordsRegister[2], coordsRegister[3], coordsRegister[4], coordsRegister[5]);
    within {
        // check if valid rectangle
        ControlledOnBitString([false, false], X)([large_start_x, large_start_y], validRectangle);
        ControlledOnBitString([true, true], X)([large_start_x, large_start_y], validRectangle);
        ControlledOnBitString([false, false], X)([large_end_x, large_end_y], validRectangle);
        ControlledOnBitString([true, true], X)([large_end_x, large_end_y], validRectangle);
        
        // check if large and small ships overlap
        EqualityOracle_2bit([small_x, small_y],[large_start_x, large_start_y], hasOverlap);
        EqualityOracle_2bit([small_x, small_y],[large_end_x, large_end_y], hasOverlap);
    }
    apply {
        // check false because hasOverlap remains at false when no overlap occurs
        ControlledOnBitString([true, false], X)([validRectangle, hasOverlap], target);
    }
}

// Implemented in Workbook_GraphColoring.ipynb
// Helper function that checks if two 2bit qubits are the same
operation EqualityOracle_2bit (small_coords : Qubit[], large_coords : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    use a = Qubit[2];
    within {
        // Compute bitwise XOR of small_coords and large_coords and store it in a
        CNOT(small_coords[0], a[0]);
        CNOT(small_coords[1], a[1]);
        CNOT(large_coords[0], a[0]);
        CNOT(large_coords[1], a[1]);
    } apply {
        // If all XORs are 0, small_coords = large_coords, and our function is 1
        (ControlledOnInt(0, X))(a, target);
    }
}

ValidBattleshipMarkingOracle, EqualityOracle_2bit

We may also test our battleship marking oracle by inputting an integer that is converted to a register of six qubits (in BigEndian order) for the oracle. For example, our test case of `inputNum=18` is converted in the following manner:

$$
18 = \textbf{0} \cdot 2^5 + \textbf{1} \cdot 2^4 + \textbf{0} \cdot 2^3 + \textbf{0} \cdot 2^2 + \textbf{1} \cdot 2^1 + \textbf{0} \cdot 2^0 \implies [|0\rangle, |1\rangle, |0\rangle, |0\rangle, |1\rangle, |0\rangle].
$$

Then, the target result is displayed through `DumpRegister`, where $|0\rangle$ represents an invalid configuration and $|1\rangle$ represents a valid configuration. In the provided test case, we have an invalid configuration since both coordinates of the $2\times 1$ ship are identical, giving a collapsed $1\times 1$ ship instead of the proper $2\times 1$ ship (see the output below where $ [\text{inputBits}[2], \text{inputBits}[3]] = [\text{inputBits}[4], \text{inputBits}[5]] = [\text{True}, \text{False}] \iff [1, 0]$). Of course, the user may try any `inputNum` between `0` and `63`.


In [64]:
operation testBattleShipOracle (inputNum : Int) : Unit {
    // current test case: 18 <=> (0, 1), ((0, 0), (1, 0))
    let inputBits = IntAsBoolArray(inputNum, 6);
    use inputRegister = Qubit[6];
    for i in 0..5 {
        Message($"inputBits[{i}]:\t{inputBits[i]}");
        if inputBits[i] {
            X(inputRegister[i]);
        }
    }
    use target = Qubit();
    ValidBattleshipMarkingOracle(inputRegister, target);
    DumpRegister((), [target]);
    ResetAll(inputRegister);
    Reset(target);
}

testBattleShipOracle

In [67]:
%simulate testBattleShipOracle inputNum=18

inputBits[0]:	False
inputBits[1]:	False
inputBits[2]:	True
inputBits[3]:	False
inputBits[4]:	True
inputBits[5]:	False


|0⟩	1 + 0𝑖
|1⟩	0 + 0𝑖

()

`target` being in the $|0\rangle$ state is consistent with the fact that this configuration of ships is invalid.

### Part 3. Using Grover's Algorithm

Now, we use the oracle implemented in the previous part as part of a Grover search. The end result should yield high amplitudes for basis states representing a valid Battleship configuration, and near-zero amplitudes otherwise. Therefore, when we perform a measurement in the computational basis, we should expect to obtain one of the states that represents a valid configuration of ships.

In the `OracleConverter` operation below, we convert the previously implemented marking oracle into a phase oracle. This allows us to flip the phases of the marked states so that `GroverAlgorithmLoop` afterwards can repeatedly perform mean inversion to amplify their probability amplitudes.

In [72]:
// Oracle Converter, GroverAlgorithmLoop, and GroversAlgorithm implemented in Graph Coloring Katas.

// Convert above marking oracle to a phase oracle, using phase-kickback

operation OracleConverter (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[]) : Unit is Adj {
    use target = Qubit();
    within {
        // Put the target qubit in the |-⟩ state
        X(target);
        H(target);
    } apply {
        // Apply the marking oracle
        markingOracle(register, target);
    }
}

operation GroverAlgorithmLoop (markingOracle : ((Qubit[], Qubit) => Unit is Adj), register : Qubit[], iterations : Int) : Unit is Adj {
    // Convert the marking oracle in a phase oracle
    let phaseOracle = OracleConverter(markingOracle, _);
    // Prepare an equal superposition of all basis states
    ApplyToEachA(H, register);
    // Apply Grover iterations
    for _ in 1..iterations {
        // Apply phase oracle
        phaseOracle(register);
        // Apply "reflection about the mean"
        within {
            ApplyToEachA(H, register);
            ApplyToEachA(X, register);
        } apply {
            (Controlled Z)(Most(register), Tail(register));
        }
    }
}

OracleConverter, GroverAlgorithmLoop

In `GroversAlgorithm`, we run `GroverAlgorithmLoop` for a certain number of iterations so that a measurement of a  marked output state returns `One`, in which case we have found our valid ship configuration. Then, `MeasureConfig` measures the configuration to extract the Boolean array of ship coordinates.

In [86]:
operation GroversAlgorithm (oracle : ((Qubit[], Qubit) => Unit is Adj)) : Bool[] {
    mutable config = new Bool[6];
    use (register, output) = (Qubit[6], Qubit());
    mutable correct = false;
    mutable iterations = 1;
    repeat {
        //Message($"Trying iteration {iterations}");
        GroverAlgorithmLoop(oracle, register, iterations);
        let temp = MultiM(register);
        oracle(register, output);
        if (MResetZ(output) == One) {
            set correct = true;
            set config = MeasureConfig(register);
        }
        ResetAll(register);
    }
    until (correct or iterations > 10)
    fixup {
        set iterations += 1;
    }
    if (not correct) {
        fail "Not a valid battleship configuration.";
    }
    return config;
}

operation MeasureConfig (register : Qubit[]) : Bool[]
{
    let measurements = MultiM(register);
    return ResultArrayAsBoolArray(measurements);
}

GroversAlgorithm, MeasureConfig

The above algorithm will only return 1 valid ship configuration, so we create an operation to run the algorithm as many times as a user wants to generate more valid ship configurations:

In [89]:
operation runGroverBattleship(num_iters : Int) : (Bool, Bool, Bool, Bool, Bool, Bool)[] {
    mutable runs = new (Bool, Bool, Bool, Bool, Bool, Bool)[num_iters];
    for n in 0..(num_iters - 1) {
        let validConfig = GroversAlgorithm(ValidBattleshipMarkingOracle);        
        set runs w/= n <- (validConfig[0], validConfig[1], validConfig[2], validConfig[3], validConfig[4], validConfig[5]);
    }
    return runs;
}

runGroverBattleShip

In [90]:
%simulate runGroverBattleship num_iters=10

(True, True, False, False, True, False), (False, False, True, True, False, True), (True, True, True, False, False, False), (False, False, True, True, False, True), (False, True, False, False, True, False), (True, True, True, False, False, False), (True, False, True, True, False, True), (False, True, True, False, True, True), (False, True, False, False, True, False), (True, False, False, True, True, True)

These are all valid ship configurations - for example, $(\text{False}, \text{True}, \text{True}, \text{False}, \text{True}, \text{True})$ corresponds to ship coordinates of $(1, 1), ([0, 0], [1, 0])$, as visualized below: 

<img src="https://user-images.githubusercontent.com/42923017/114302423-f3bdb480-9a96-11eb-8dab-07be87d86a7f.png" alt="Richard's Battleship" width="300" caption="Valid Battleship"/>

This project was a great experience in seeing first-hand how Grover's Algorithm can be utilized to solve for the satisfiability of $2\times 2$ battleship grid and implementing yet another exciting quantum oracle. It would be interesting to explore quantum oracles for battleship grids of higher dimensions and start being able to guess with high probability where the ships in other battleship grids lie.